In [23]:
from __future__ import print_function  # Only needed for Python 2
import pickle
from collections import defaultdict 
import json
import codecs

proxies = {
    'HTTP': 'http://37.235.67.178:34450',
    'HTTP': 'http://109.201.97.204:41258',
    'HTTP': 'http://37.29.48.78:8080',
    'HTTP': 'http://85.237.57.198:42118',
    'HTTP': 'http://31.133.121.28:31315',
    'HTTP': 'http://95.31.4.20:54618',
}

def store_parsing_stage(s):
    with open('parsing_stage.pickle', 'wb') as f:
        pickle.dump(s.parsing_stage, f)
        
def restore_parsing_stage():
    with open('parsing_stage.pickle', 'rb') as f:
        parsing_stage = pickle.load(f)
        return parsing_stage
    return {'row':0, 'page':1}

def store_dict(s):
    with codecs.open('reader_dict.json', 'w', encoding='utf-8') as f:
        json.dump(dict(s.reader_dict), f, sort_keys=True,
                  indent=4, separators=(',', ': '), ensure_ascii=False)
        
def restore_dict():
    with codecs.open('reader_dict.json', 'r', encoding='utf-8') as f:
        reader_dict = json.load(f)
        return defaultdict(list, reader_dict)
    return defaultdict(list)

def store(s):
    store_dict(s)
    store_parsing_stage(s)


def update_parsing_stage(s):
    s.parsing_stage['page'] = 1
    s.parsing_stage['row'] += 1

class Serialized:
    def deserialize(self):
        self.parsing_stage = restore_parsing_stage()
        self.reader_dict = restore_dict()
        
def make_readers_dict(readers):
    rdict = {}
    for idx, r in enumerate(readers.itertuples()):
        rdict[r.ссылка] = r
    return rdict

In [19]:
import requests
import html2text
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from re import sub 

def clear_sentence(s):
    return sub(r'[\.\—\?\!\n\t\,\«\»\)\(\:\/\=]',' ',s)

bad_words=['спойлер','свернуть']
def preprocessed_text(s):
    def erase_space_symbols(s):
        return sub(r'[\t\n]',' ', s)
    def erase_spec_symbols(s):
        return sub(r'[\>\<\_\*]', '', s)
    def erase_links(s):
        return sub(r'\[.{,70}\]\(.{,200}\)', 'ССЫЛКА', s)
    for w in bad_words:
        s = s.replace(w,'')
    s = erase_space_symbols(s)
    s = erase_spec_symbols(s)
    s = erase_links(s)
    return s

def parse_reader(reader, s):
    url_base = 'https://www.livelib.ru/reader/' + reader.ссылка + '/reviews/~'
    review_count = 0
    for page_number in range(s.parsing_stage['page'], 2):
        url = url_base + str(page_number)
        req = requests.get(url)
        if not req:
            print('bad req ' + url)
            sleep(11)
            break
        req.encoding = 'utf-8'
        soup = BeautifulSoup(req.text, "lxml")
        html = soup.findAll('div', itemprop='reviewBody')
        response_len = len(html)
        if not response_len:
            if 'Капча' in req.text:
                print(req.text)
                store(s)
                raise
            else:
                update_parsing_stage(s)
                break
        review_count += response_len
        for idx, div in enumerate(html):
            text = html2text.html2text(str(div))
            text = preprocessed_text(text)
            s.reader_dict[reader.ссылка].append(text)
        print('len ' + str(len(s.reader_dict[reader.ссылка])))

        s.parsing_stage['page'] += 1
#         store(s)
        sleep(11)
    update_parsing_stage(s)


def reset(s):
    s.parsing_stage = {'row':0,'page':1}
    store_parsing_stage(s)
    s.reader_dict = defaultdict(list)
    store_dict(s)
    
s = Serialized()
reset(s)
s.deserialize()

readers = pd.read_csv('readers.csv')

for idx, r in enumerate(readers.itertuples()):
    if idx < s.parsing_stage['row']:
        continue
    print('idx ' + str(idx))
    parse_reader(r,s)
    if not idx % 5:
        store(s)

print('FINISHED')

idx 0
len 25
idx 1
len 25
idx 2
len 25
idx 3
len 25
idx 4
len 25
idx 5
len 25
idx 6
len 25
idx 7
len 25
idx 8
len 25
idx 9
len 25
idx 10
len 25
idx 11
len 25
idx 12
len 25
idx 13
len 25
idx 14
len 25
idx 15
len 15
idx 16
len 25
idx 17
len 25
idx 18
len 22
idx 19
len 25
idx 20
len 25
idx 21
len 25
idx 22
len 25
idx 23
len 25
idx 24
len 25
idx 25
len 25
idx 26
len 25
idx 27
len 5
idx 28
idx 30
len 25
idx 31
len 25
idx 32
len 25
idx 33
len 25
idx 34
len 6
idx 35
len 25
idx 36
len 25
idx 37
len 10
idx 38
len 25
idx 39
len 25
idx 40
len 25
idx 41
len 2
idx 42
len 13
idx 43
len 25
idx 44
len 25
idx 45
len 25
idx 46
len 2
FINISHED


In [34]:
# split corpus by tags

rdict = make_readers_dict(readers)

# TAGS
genders = ['ж', 'м']
years = ['<18', '18-27','28-40', '>40']
level = ['-', 'учитель', 'писатель']
mentality = ['-', 'гум', 'тех']


def fill_file(rmatcher, fname):
    with codecs.open('splitted_corpora/' + fname, 'w', encoding='utf-8') as f:
        for i in range(0,25):
            for key, reader in rdict.items():
                if rmatcher(reader):
                    reviews = s.reader_dict[key]
                    if i < len(reviews):
                        print(reviews[i], file=f)

# matchers
f_matcher = lambda x: x.Пол == 'ж'
m_matcher = lambda x: x.Пол == 'м'        

yl18_matcher = lambda x: x.Год > 2000
y18_27_matcher = lambda x: 1991 <= x.Год <= 2000
y28_40_matcher = lambda x: 1978 <= x.Год <= 1991
yg40_matcher = lambda x: x.Год > 1978

no_level_matcher = lambda x: x.Профессия == '-'
teacher_matcher = lambda x:  x.Профессия == 'учитель'
writer_matcher = lambda x:   x.Профессия == 'писатель'

no_mentality_mantcher = lambda x: x.гумтех == 'без'
humanist_matcher = lambda x: x.гумтех == 'гум'
techie_matcher = lambda x:   x.гумтех == 'тех'

tag_dict = {
    'м': {
        'file':'males.txt',
        'matcher':m_matcher,
        'value':[0]
    },
    'ж': {
        'file':'femalies.txt',
        'matcher':f_matcher,
        'value':[1]
    },
    'моложе_18': {
        'file':'younger_18.txt',
        'matcher':yl18_matcher,
        'value': [0,0,0,1]
    },
    '18_27': {
        'file':'from_18_to_27.txt',
        'matcher':y18_27_matcher,
        'value':[0,0,1,0]
    },
    '28_40': {
        'file':'from_27_to_40.txt',
        'matcher':y28_40_matcher,
        'value':[0,1,0,0]
    },
    'старше_40': {
        'file':'older_41.txt',
        'matcher':yg40_matcher,
        'value':[1,0,0,0]
    },
    'неизв_проф': {
        'file':'unknown_level.txt',
        'matcher':no_level_matcher,
        'value':[0,0,1]
    },
    'учитель': {
        'file':'teachers.txt',
        'matcher':teacher_matcher,
        'value0':[0,1,0]
    },
    'писатель': {
        'file':'writers.txt',
        'matcher':writer_matcher,
        'value':[1,0,0]
    },
    'неизв_склад': {
        'file':'unknown_mentality.txt',
        'matcher':no_mentality_mantcher,
        'value':[0,0,1]
    },
    'гуманитарий': {
        'file':'humanists.txt',
        'matcher':humanist_matcher,
        'value':[0,1,0]
    },
    'технарь': {
        'file':'techies.txt',
        'matcher':techie_matcher,
        'value':[1,0,0]
    },
    
}

# fill files
for key, tag in tag_dict.items():
    fill_file(tag['matcher'], tag['file'])


print('FINISHED')

FINISHED
